<h1>Twitter sentiment analysis</h1>

---> To get High Sentiment Tweet

---> To get Low Sentiment Tweet

---> To get Most Retweeted Tweet

---> To get User Producing most Tweets



In [3]:
import html
import json
import string
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
#Collect relevant tweets through the Twitter API.
import json
import tweepy as tw

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h2>Connecting to Twitter API</h2>

In [4]:
#IMPORTANT: enter proper access credential in config_twitter.py file
import config_twitter

In [5]:
#function to establish an initial API connection, respecting the rate limit
def connect_api_client():
    auth = tw.OAuthHandler(config_twitter.CONSUMER_KEY, config_twitter.CONSUMER_SECRET)
    auth.set_access_token(config_twitter.ACCESS_TOKEN, config_twitter.ACCESS_SECRET)
    api = tw.API(auth, wait_on_rate_limit=True)
    try:
        # returns False if credentials could not be verified
        api.verify_credentials()
        user = api.verify_credentials()
        if not user:
            raise("Credentials could not be verified: Please check config.py")
        print(f"Connected to Twitter API as {user.name}")
    except Exception as e:
        raise e
    return api

In [6]:
api = connect_api_client()

Connected to Twitter API as Ganesh Kumaran Masilamani


<h2>Collecting Data for Charity</h2>



In [7]:
#construct a search query
query = 'charity OR "charity donation" OR "spend charity" OR "pay charity" -filter:retweets'
#decide how many tweets to query
ntweets = 2000
#search and collect relevant tweets
tweets = [tweet._json for tweet in tw.Cursor(api.search, q=query, lang="en", tweet_mode='extended').items(ntweets)]
len(tweets)

2000

In [8]:
#example tweet content (json structure)
tweets[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Aug 10 22:36:22 +0000 2021',
 'display_text_range': [20, 192],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 351955341,
    'id_str': '351955341',
    'indices': [0, 8],
    'name': 'Trevor Stone',
    'screen_name': 'tms1967'},
   {'id': 248741839,
    'id_str': '248741839',
    'indices': [9, 19],
    'name': 'Tim Dunn',
    'screen_name': 'MrTimDunn'}]},
 'favorite_count': 0,
 'favorited': False,
 'full_text': '@tms1967 @MrTimDunn Got them! A very nice set turned up in a local charity shop last year. I briefly worked with JHT as a teenager, on the abortive railway museum in Crystal Palace goods yard.',
 'geo': None,
 'id': 1425224530012803075,
 'id_str': '1425224530012803075',
 'in_reply_to_screen_name': 'tms1967',
 'in_reply_to_status_id': 1425218524264423426,
 'in_reply_to_status_id_str': '1425218524264423426',
 'in_reply_to_user_id': 351955341,
 'in_reply_to_user_id_str': '3

In [9]:
#save tweets data to json file
file_out = f"raw_tweet_data_charities_{ntweets}.json"
with open(file_out, mode='w') as f:
    f.write(json.dumps(tweets, indent=2))
#First collect the data in json-file; specify file name here (adjust the number as queried)
fjson = 'raw_tweet_data_charities_2000.json'

In [10]:
#read json file with tweets data
with open(fjson) as file:
    data = json.load(file)
len(data)

2000

In [11]:
#tweet data record example: as documented for the Twitter API
data[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Aug 10 22:36:22 +0000 2021',
 'display_text_range': [20, 192],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 351955341,
    'id_str': '351955341',
    'indices': [0, 8],
    'name': 'Trevor Stone',
    'screen_name': 'tms1967'},
   {'id': 248741839,
    'id_str': '248741839',
    'indices': [9, 19],
    'name': 'Tim Dunn',
    'screen_name': 'MrTimDunn'}]},
 'favorite_count': 0,
 'favorited': False,
 'full_text': '@tms1967 @MrTimDunn Got them! A very nice set turned up in a local charity shop last year. I briefly worked with JHT as a teenager, on the abortive railway museum in Crystal Palace goods yard.',
 'geo': None,
 'id': 1425224530012803075,
 'id_str': '1425224530012803075',
 'in_reply_to_screen_name': 'tms1967',
 'in_reply_to_status_id': 1425218524264423426,
 'in_reply_to_status_id_str': '1425218524264423426',
 'in_reply_to_user_id': 351955341,
 'in_reply_to_user_id_str': '3

In [12]:
#create pandas dataframe from tweet text content
df_tweets = pd.DataFrame([t['full_text'] for t in data], columns=['text'])
df_tweets

,text
0,@tms1967 @MrTimDunn Got them! A very nice set ...
1,Pray in tongues a lot.
2,About the whole Matt Caster thing: if he had s...
3,@rhymingmisfit @JoeGrowling I work for a chari...
4,#Quran \n\n O #believers ! Do not make your #c...
...,...
1995,@hike_charity @tina_swindells Awe bless...Nigh...
1996,"Hey its my first ever charity stream, let's sa..."
1997,@alanhendrick @piersmorgan @Cristiano He does ...
1998,Jon Stewart And Pete Davidson Are Doing A 9/11...


In [13]:
#add selected columns from tweet data fields
df_tweets['retweets'] = [t['retweet_count'] for t in data]
df_tweets['favorites'] = [t['favorite_count'] for t in data]
df_tweets['user'] = [t['user']['screen_name'] for t in data]
df_tweets

,text,retweets,favorites,user
0,@tms1967 @MrTimDunn Got them! A very nice set ...,0,0,cmajrail
1,Pray in tongues a lot.,0,0,Charity_Heals
2,About the whole Matt Caster thing: if he had s...,0,0,WrestleTalk15
3,@rhymingmisfit @JoeGrowling I work for a chari...,0,0,BRUMSTOKIE
4,#Quran \n\n O #believers ! Do not make your #c...,0,0,fahim_venus
...,...,...,...,...
1995,@hike_charity @tina_swindells Awe bless...Nigh...,0,1,itsmemorna
1996,"Hey its my first ever charity stream, let's sa...",4,5,RedBeardUnleash
1997,@alanhendrick @piersmorgan @Cristiano He does ...,0,0,KASFFC
1998,Jon Stewart And Pete Davidson Are Doing A 9/11...,0,0,RbySlprEstate


In [14]:
stop_words = set(stopwords.words('english'))
def text_cleanup(s):
    s_unesc = html.unescape(re.sub(r"http\S+", "", re.sub('\n+', ' ', s)))
    s_noemoji = s_unesc.encode('ascii', 'ignore').decode('ascii')
    #normalize to lowercase and tokenize
    wt = word_tokenize(s_noemoji.lower())
    #filter word-tokens
    wt_filt = [w for w in wt if (w not in stop_words) and (w not in string.punctuation) and (w.isalnum())]
    #return clean string
    return ' '.join(wt_filt)

In [15]:
#add clean text column
#NOTE: apply in pandas applies a function to each element of the selected column
df_tweets['text_clean'] = df_tweets['text'].apply(text_cleanup)
df_tweets

,text,retweets,favorites,user,text_clean
0,@tms1967 @MrTimDunn Got them! A very nice set ...,0,0,cmajrail,tms1967 mrtimdunn got nice set turned local ch...
1,Pray in tongues a lot.,0,0,Charity_Heals,pray tongues lot
2,About the whole Matt Caster thing: if he had s...,0,0,WrestleTalk15,whole matt caster thing said random dark episo...
3,@rhymingmisfit @JoeGrowling I work for a chari...,0,0,BRUMSTOKIE,rhymingmisfit joegrowling work charity deals m...
4,#Quran \n\n O #believers ! Do not make your #c...,0,0,fahim_venus,quran believers make charity worthless reminde...
...,...,...,...,...,...
1995,@hike_charity @tina_swindells Awe bless...Nigh...,0,1,itsmemorna,awe bless night dave
1996,"Hey its my first ever charity stream, let's sa...",4,5,RedBeardUnleash,hey first ever charity stream let save rainbows
1997,@alanhendrick @piersmorgan @Cristiano He does ...,0,0,KASFFC,alanhendrick piersmorgan cristiano much charit...
1998,Jon Stewart And Pete Davidson Are Doing A 9/11...,0,0,RbySlprEstate,jon stewart pete davidson anniversary comedy s...


In [16]:
df_tweets.to_csv (r'Charities_tweets.csv', index = False, header=True)

<h2>Highest Sentiment Tweets</h2>



In [17]:
#sentiment analysis
def sentim_polarity(s):
    return TextBlob(s).sentiment.polarity
def sentim_subject(s):
    return TextBlob(s).sentiment.subjectivity
df_tweets['polarity'] = df_tweets['text_clean'].apply(sentim_polarity)
df_tweets['subjectivity'] = df_tweets['text_clean'].apply(sentim_subject)
df_tweets

,text,retweets,favorites,user,text_clean,polarity,subjectivity
0,@tms1967 @MrTimDunn Got them! A very nice set ...,0,0,cmajrail,tms1967 mrtimdunn got nice set turned local ch...,0.150000,0.350000
1,Pray in tongues a lot.,0,0,Charity_Heals,pray tongues lot,0.000000,0.000000
2,About the whole Matt Caster thing: if he had s...,0,0,WrestleTalk15,whole matt caster thing said random dark episo...,-0.295000,0.580000
3,@rhymingmisfit @JoeGrowling I work for a chari...,0,0,BRUMSTOKIE,rhymingmisfit joegrowling work charity deals m...,0.183333,0.633333
4,#Quran \n\n O #believers ! Do not make your #c...,0,0,fahim_venus,quran believers make charity worthless reminde...,-0.400000,0.483333
...,...,...,...,...,...,...,...
1995,@hike_charity @tina_swindells Awe bless...Nigh...,0,1,itsmemorna,awe bless night dave,0.000000,0.000000
1996,"Hey its my first ever charity stream, let's sa...",4,5,RedBeardUnleash,hey first ever charity stream let save rainbows,0.250000,0.333333
1997,@alanhendrick @piersmorgan @Cristiano He does ...,0,0,KASFFC,alanhendrick piersmorgan cristiano much charit...,0.133333,0.166667
1998,Jon Stewart And Pete Davidson Are Doing A 9/11...,0,0,RbySlprEstate,jon stewart pete davidson anniversary comedy s...,0.000000,0.000000


In [18]:
#Highest sentiment tweets
df_tweets.sort_values(by='polarity', ascending=False).head(20)

,text,retweets,favorites,user,text_clean,polarity,subjectivity
1985,⛰️ZIPWIRE CHALLENGE 2021⛰️\n\nCome join One Um...,0,0,OneUmmahCharity,zipwire challenge 2021 come join one ummah cha...,1.000,1.000000
1369,Psst...\nWant to be in with a chance of owning...,2,4,prosaic97,psst want chance owning gem painting wonderful...,1.000,1.000000
1640,@brian_2436 @itsprettyjamie @JoanaTiw Awesome ...,0,0,komal30591,itsprettyjamie joanatiw awesome nfts combined ...,1.000,1.000000
1106,@yo @coolcatsnft Look into @TopDogBeachClub if...,1,19,DoseETH,yo coolcatsnft look topdogbeachclub looking be...,1.000,0.300000
1530,I did this thing at a charity best ball tourne...,0,1,SportsLawBuff,thing charity best ball tourney one time close...,1.000,0.300000
615,@DaCryptoGeneral $SAVE @stwtoken best charity/...,0,1,kamelenyoghurt,dacryptogeneral save stwtoken best investment ...,1.000,0.300000
1161,@iAmDrSoup @StevieJRaw @spirituallyemp1 @ComeG...,0,0,shawme101,iamdrsoup steviejraw spirituallyemp1 comegeeks...,1.000,0.300000
1414,@OperHealAmerica 1st step is faith. But the ac...,0,0,BedardSebastien,operhealamerica 1st step faith accomplishment ...,1.000,1.000000
1441,@bingus_network The best charity token ever,0,1,chase_oak,best charity token ever,1.000,0.300000
1208,$500 in @canadahelps charity gift cards is up ...,0,0,hollymTO,500 canadahelps charity gift cards grabs lil i...,1.000,1.000000


<h2>Lowest Sentiment Tweets</h2>



In [19]:
#Lowest sentiment tweets
df_tweets.sort_values(by='polarity', ascending=True).head(20)

,text,retweets,favorites,user,text_clean,polarity,subjectivity
1920,@SWTheory66 I'll be there Theory with some mon...,0,0,agentnightron,swtheory66 theory money donate ewan insane lik...,-1.000000,1.000000
574,Might as well give the £50k to charity at this...,0,2,JustSimplyEjan,might well give 50k charity point itv2 loveisl...,-1.000000,1.000000
191,@EricHaftelLive @WontCaveDave Both @GregAbbott...,0,1,GregMercerMSN,erichaftellive wontcavedave govrondesantis pub...,-1.000000,1.000000
70,@LovecraftLuci @FluffySunshine6 A charity that...,0,1,bullrike,lovecraftluci fluffysunshine6 charity chides t...,-0.900000,1.000000
577,@ilucero1028 @people It literally says she’s d...,0,2,itsjeffepstein,ilucero1028 people literally says shes donatin...,-0.800000,0.900000
66,@outandaboutjc1 @ToddHagopian They do sell tic...,0,0,LibertarianJedi,outandaboutjc1 toddhagopian sell tickets stupi...,-0.800000,1.000000
1615,"Can you imagine if Dorit, Rinna &amp; Kyle wer...",0,7,ShadeRealitea,imagine dorit rinna kyle petty vicious erika l...,-0.800000,0.777778
312,@Charity_AG Oh no! That's so awful! I'm so sor...,0,1,VoMylo,oh awful sorry happened hang ca imagine feels,-0.750000,1.000000
1772,@DonaldJTrumpJr Almost as sick and depraved as...,0,0,shannonswg,donaldjtrumpjr almost sick depraved stealing m...,-0.714286,0.857143
297,you ever sat somewhere and think someone smell...,0,1,franzesty,ever sat somewhere think someone smells like c...,-0.700000,0.666667


<h2>Most Retweeted Content</h2>



In [20]:
#most retweeted content
df_tweets.sort_values(by='retweets', ascending=False).head(20)

,text,retweets,favorites,user,text_clean,polarity,subjectivity
1288,Give this 50k to charity.\n\n#LoveIsland https...,269,2093,Gurdz2012,give 50k charity loveisland,0.000000,0.000000
1127,Just give the money to charity #loveIsland,149,679,_azryah,give money charity loveisland,0.000000,0.000000
1974,Had a zoom call with St Jude's charity. They w...,120,1293,SWTheory66,zoom call st jude charity want big charity eve...,0.000000,0.083333
1526,Beeduo with Micheal illustration I did for @Dr...,70,431,OwllekArt,beeduo micheal illustration dromserver charity...,0.000000,0.000000
1270,Have you seen the new trans equality design? T...,53,471,WholesomeMeme,seen new trans equality design designated bene...,0.136364,0.454545
402,Ranboo said that he's planning to do another c...,50,2933,RanbooUpdates,ranboo said planning another charity subathon ...,0.000000,0.000000
103,$350k+ $SUSHI Paid in 5 hours!\n\nWe've set a ...,44,67,zooshicoin,sushi paid 5 hours set goal hit within 48 hour...,0.267857,0.434524
1673,"I'm hollering\n\nAnti on melOn: ""Hyolyn isn't ...",34,71,photo_jenic,hollering anti melon hyolyn making money soloi...,-0.333333,0.833333
381,Ranboo just announced that he is going to be d...,30,453,mctwtpaper,ranboo announced going another charity subatho...,-0.133333,0.133333
1125,Please give the 50K to charity #LoveIsland,27,70,Ging_CasCas,please give 50k charity loveisland,0.000000,0.000000


<h2>Users Producing Most Content</h2>



In [21]:
#users producing most retweeted content
df_tweets.sort_values(by='retweets', ascending=False).head(20)['user']

1288         Gurdz2012
1127           _azryah
1974        SWTheory66
1526         OwllekArt
1270     WholesomeMeme
402      RanbooUpdates
103         zooshicoin
1673       photo_jenic
381         mctwtpaper
1125       Ging_CasCas
467     benchtwtupdate
1412          raxifire
717           Strat_AI
1544       loverofjoy_
1465     RockLordsRock
196       ranbooupdate
1525      sandicon2022
408       alliumupdate
1296            Sirovi
1245        grass_pale
Name: user, dtype: object